In [28]:
# notebooks/02_build_labels.ipynb (conceptual cells)
import sys
sys.path.insert(0, '../src')

from insider.clients.prices_yf import download_adj_close
from insider.clients.market_cap_yf import get_market_caps
from insider.calendar import nyse_days
from insider.features import compute_forward_return, log1p_safe, size_vs_cap
from insider.config import SETTINGS
import pandas as pd
import numpy as np



# events_2025Q1.parquet is the jopined data obtained after mergest relevant data from Transactions, Submissions and Report Owner
events = pd.read_parquet("data/raw/events_2025Q1.parquet")  
events.head()

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1
0,0000002488-25-000014,buy,499616.20,2025-02-07,13.121597,OFFICER,AMD,0
1,0000002488-25-000016,sell,1987545.16,2025-02-14,14.502411,OFFICER,AMD,1
2,0000002488-25-000034,sell,1754674.32,2025-03-14,14.377794,OFFICER,AMD,1
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0


In [29]:
tickers = ['^GSPC'] + sorted(events["ticker"].unique())
print(len(tickers))

start = events["date"].min() - pd.Timedelta(days=10)    # -10 cushion for non-trading days
end   = events["date"].max() + pd.Timedelta(days=180)   # +180 ensure data for t+63
print(f"start={start}, end={end}")

2374
start=2024-12-22 00:00:00, end=2025-09-27 00:00:00


## Fetch stock prices

In [30]:
#start = '2025-01-01'
#end = '2025-01-15'
#tickers = ['^GSPC', 'MSFT', 'META', 'GOOGL']
#px = download_adj_close(tickers, start, end)

In [31]:
# Data fetch happened on 11-24-2025. Number of columns with any NaN: 1189 out of 2374
# px.to_parquet("data/engineered/price_data_2025Q1.parquet") 
# price_data_2025Q1.parquet stores the fetched stock market data
px = pd.read_parquet("data/engineered/price_data_2025Q1.parquet")  
print(f"px shape: {px.shape}")
print(f"Number of columns with any NaN: {px.isnull().any().sum()}")

# Drop all columns with any NaN
px = px.dropna(axis=1, how='any')
print(f"px shape after dropping columns with NaN: {px.shape}")

px shape: (190, 2372)
Number of columns with any NaN: 1189
px shape after dropping columns with NaN: (190, 1183)


In [32]:
td = nyse_days(start, end)

events = compute_forward_return(events, px, td, n=63)
events = events.dropna(subset=["P_t","P_t_plus_63"])

events["forward_ret_63"] = events["P_t_plus_63"]/events["P_t"] - 1
events["label_up"] = (events["forward_ret_63"] > 0).astype(int)


In [33]:
events.head()

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1,P_t,P_t_plus_63,forward_ret_63,label_up
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0,217.482407,232.501556,0.069059,1
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0,218.415924,230.700226,0.056243,1
5,0000004127-25-000028,buy,661300.00,2025-02-25,13.401964,OFFICER,SWKS,0,64.137993,69.934570,0.090377,1
6,0000004977-25-000049,sell,2357419.89,2025-03-07,14.673079,OFFICER,AFL,1,106.673180,102.582268,-0.038350,0
7,0000004977-25-000051,sell,107630.00,2025-03-07,11.586464,DIRECTOR,AFL,1,106.673180,102.582268,-0.038350,0


## Add market benchmark with S&P500

In [34]:
# Add SP500 (^GSPC) forward returns to events
# Get SP500 prices at event date and t+63

def get_sp500_prices(row):
    """Get SP500 prices at transaction date and 63 days later"""
    t = pd.Timestamp(row["date"])
    
    # Find SP500 price at or nearest to transaction date
    sp500_prices = px['^GSPC'].dropna()
    pos_t = sp500_prices.index.searchsorted(t, "left")
    if pos_t >= len(sp500_prices):
        pos_t = len(sp500_prices) - 1
    sp500_t = sp500_prices.iloc[pos_t]
    
    # Find SP500 price 63 trading days later
    # Estimate position (roughly 63 trading days ≈ 90 calendar days)
    t_plus_63_approx = t + pd.Timedelta(days=90)
    pos_t63 = sp500_prices.index.searchsorted(t_plus_63_approx, "left")
    if pos_t63 >= len(sp500_prices):
        pos_t63 = len(sp500_prices) - 1
    sp500_t63 = sp500_prices.iloc[pos_t63]
    
    if pd.isna(sp500_t) or pd.isna(sp500_t63) or sp500_t == 0:
        return pd.Series({"SP500_t": np.nan, "SP500_t_plus_63": np.nan, "forward_ret_SP500": np.nan})
    
    forward_ret_sp500 = sp500_t63 / sp500_t - 1
    return pd.Series({"SP500_t": float(sp500_t), "SP500_t_plus_63": float(sp500_t63), "forward_ret_SP500": forward_ret_sp500})

# Apply the function
sp500_data = events.apply(get_sp500_prices, axis=1)
events = pd.concat([events, sp500_data], axis=1)

print("Added SP500 columns:")
print(events[['date', 'SP500_t', 'SP500_t_plus_63', 'forward_ret_SP500']].head())

Added SP500 columns:
        date      SP500_t  SP500_t_plus_63  forward_ret_SP500
3 2025-02-26  5956.060059      5921.540039          -0.005796
4 2025-02-27  5861.569824      5888.549805           0.004603
5 2025-02-25  5955.250000      5921.540039          -0.005661
6 2025-03-07  5770.200195      5939.299805           0.029306
7 2025-03-07  5770.200195      5939.299805           0.029306


In [35]:
events.head()

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1,P_t,P_t_plus_63,forward_ret_63,label_up,SP500_t,SP500_t_plus_63,forward_ret_SP500
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0,217.482407,232.501556,0.069059,1,5956.060059,5921.540039,-0.005796
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0,218.415924,230.700226,0.056243,1,5861.569824,5888.549805,0.004603
5,0000004127-25-000028,buy,661300.00,2025-02-25,13.401964,OFFICER,SWKS,0,64.137993,69.934570,0.090377,1,5955.250000,5921.540039,-0.005661
6,0000004977-25-000049,sell,2357419.89,2025-03-07,14.673079,OFFICER,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306
7,0000004977-25-000051,sell,107630.00,2025-03-07,11.586464,DIRECTOR,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306


In [36]:
# Add label_up_market: 1 if stock outperforms SP500, 0 otherwise
events["label_up_market"] = (events["forward_ret_63"] > events["forward_ret_SP500"]).astype(int)

print("Added label_up_market column:")
print(events[['forward_ret_63', 'forward_ret_SP500', 'label_up_market']].head(10))
print(f"\nLabel distribution:")
print(events['label_up_market'].value_counts())

Added label_up_market column:
    forward_ret_63  forward_ret_SP500  label_up_market
3         0.069059          -0.005796                1
4         0.056243           0.004603                1
5         0.090377          -0.005661                1
6        -0.038350           0.029306                0
7        -0.038350           0.029306                0
8        -0.034507           0.069697                0
9         0.011868           0.014739                0
10        0.011868           0.014739                0
11        0.011868           0.014739                0
19       -0.078321          -0.091873                1

Label distribution:
label_up_market
0    3596
1    2336
Name: count, dtype: int64


In [37]:
events.head(10)

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1,P_t,P_t_plus_63,forward_ret_63,label_up,SP500_t,SP500_t_plus_63,forward_ret_SP500,label_up_market
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0,217.482407,232.501556,0.069059,1,5956.060059,5921.540039,-0.005796,1
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0,218.415924,230.700226,0.056243,1,5861.569824,5888.549805,0.004603,1
5,0000004127-25-000028,buy,661300.00,2025-02-25,13.401964,OFFICER,SWKS,0,64.137993,69.934570,0.090377,1,5955.250000,5921.540039,-0.005661,1
6,0000004977-25-000049,sell,2357419.89,2025-03-07,14.673079,OFFICER,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0
7,0000004977-25-000051,sell,107630.00,2025-03-07,11.586464,DIRECTOR,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0
8,0000004977-25-000053,sell,5691295.95,2025-03-10,15.554449,OFFICER,AFL,1,105.049606,101.424652,-0.034507,0,5614.560059,6005.879883,0.069697,0
9,0000005513-25-000021,sell,579250.00,2025-03-03,13.269491,OFFICER,UNM,0,80.009109,80.958687,0.011868,1,5849.720215,5935.939941,0.014739,0
10,0000005513-25-000025,sell,4099441.70,2025-03-03,15.226362,OFFICER,UNM,0,80.009109,80.958687,0.011868,1,5849.720215,5935.939941,0.014739,0
11,0000005513-25-000035,sell,808809.57,2025-03-03,13.603320,DIRECTOR,UNM,0,80.009109,80.958687,0.011868,1,5849.720215,5935.939941,0.014739,0
19,0000016732-25-000002,sell,83800.00,2025-01-03,11.336200,OFFICER,CPB,0,40.324444,37.166210,-0.078321,0,5942.470215,5396.520020,-0.091873,1


In [38]:
print(len(events))

5932


## Fetch Market Cap

In [39]:
# Get all tickers from px (excluding the index)
tickers_list = px.columns.tolist()
#print(f"Fetching market caps for {len(tickers_list)} tickers...")


In [40]:
# Fetch market caps from API
# Fetch completed in 11/25/2025. Took ~14 minutes to run. 
# Market caps fetched: 1178 with data, 5 missing
#market_caps = get_market_caps(tickers_list)
#market_cap_df = pd.DataFrame(list(market_caps.items()), columns=['ticker', 'market_cap'])


In [41]:

# Convert to DataFrame for easier viewing
#print(f"\nMarket caps fetched: {len(market_cap_df[market_cap_df['market_cap'].notna()])} with data, {len(market_cap_df[market_cap_df['market_cap'].isna()])} missing")
#print("\nSample market caps:")
#print(market_cap_df.head(10))

#market_cap_df.to_parquet("data/engineered/market_cap_data_2025Q1.parquet")   

# Create a mapping for events
#market_cap_map = market_caps
#print(f"\nTotal tickers with market cap data: {sum(1 for v in market_cap_map.values() if v is not None)}")



In [42]:
# market_cap_data_2025Q1.parquet stores the market cap data fetched from APIs
market_cap_df = pd.read_parquet("data/engineered/market_cap_data_2025Q1.parquet")  

In [43]:
print(f"px shape: {market_cap_df.shape}")

px shape: (1183, 2)


In [44]:
# Create a mapping of ticker to market cap
market_cap_map = dict(zip(market_cap_df['ticker'], market_cap_df['market_cap']))

# Add size_vs_cap column: dollar_value / market_cap
events['market_cap'] = events['ticker'].map(market_cap_map)
events['size_vs_cap'] = events['dollar_value'] / events['market_cap']

# Replace inf and -inf with NaN
events['size_vs_cap'] = events['size_vs_cap'].replace([np.inf, -np.inf], np.nan)

print("Added size_vs_cap column:")
print(events[['ticker', 'dollar_value', 'market_cap', 'size_vs_cap']].head(10))
print(f"\nMissing values in size_vs_cap: {events['size_vs_cap'].isnull().sum()}")
print(f"Summary statistics for size_vs_cap:")
print(events['size_vs_cap'].describe())

Added size_vs_cap column:
   ticker  dollar_value    market_cap  size_vs_cap
3     LNG    5081343.54  4.496016e+10     0.000113
4     LNG    1975590.00  4.496016e+10     0.000044
5    SWKS     661300.00  9.374260e+09     0.000071
6     AFL    2357419.89  5.903470e+10     0.000040
7     AFL     107630.00  5.903470e+10     0.000002
8     AFL    5691295.95  5.903470e+10     0.000096
9     UNM     579250.00  1.288590e+10     0.000045
10    UNM    4099441.70  1.288590e+10     0.000318
11    UNM     808809.57  1.288590e+10     0.000063
19    CPB      83800.00  9.121551e+09     0.000009

Missing values in size_vs_cap: 8
Summary statistics for size_vs_cap:
count    5.924000e+03
mean     2.914681e-03
std      7.946250e-02
min      1.171792e-08
25%      1.641532e-05
50%      6.789089e-05
75%      2.806214e-04
max      4.756259e+00
Name: size_vs_cap, dtype: float64


In [45]:
events.head()

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1,P_t,P_t_plus_63,forward_ret_63,label_up,SP500_t,SP500_t_plus_63,forward_ret_SP500,label_up_market,market_cap,size_vs_cap
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0,217.482407,232.501556,0.069059,1,5956.060059,5921.540039,-0.005796,1,4.496016e+10,0.000113
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0,218.415924,230.700226,0.056243,1,5861.569824,5888.549805,0.004603,1,4.496016e+10,0.000044
5,0000004127-25-000028,buy,661300.00,2025-02-25,13.401964,OFFICER,SWKS,0,64.137993,69.934570,0.090377,1,5955.250000,5921.540039,-0.005661,1,9.374260e+09,0.000071
6,0000004977-25-000049,sell,2357419.89,2025-03-07,14.673079,OFFICER,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0,5.903470e+10,0.000040
7,0000004977-25-000051,sell,107630.00,2025-03-07,11.586464,DIRECTOR,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0,5.903470e+10,0.000002


In [46]:
print("Total rows in events:", len(events))
print("Rows with any missing or NaN values:", events.isnull().any(axis=1).sum())
print("\nNaN counts per column:")
print(events.isnull().sum())

Total rows in events: 5932
Rows with any missing or NaN values: 8

NaN counts per column:
ACCESSION_NUMBER     0
side                 0
dollar_value         0
date                 0
log_dollar_value     0
role                 0
ticker               0
is_10b5_1            0
P_t                  0
P_t_plus_63          0
forward_ret_63       0
label_up             0
SP500_t              0
SP500_t_plus_63      0
forward_ret_SP500    0
label_up_market      0
market_cap           8
size_vs_cap          8
dtype: int64


In [47]:
# Drop rows with any NaN or missing values
events = events.dropna()

# Verify the changes
print("Total rows after dropping NaN:", len(events))
print("Rows with any missing or NaN values after drop:", events.isnull().any(axis=1).sum())

Total rows after dropping NaN: 5924
Rows with any missing or NaN values after drop: 0


In [48]:
# Add natural log of size_vs_cap column
events['log_size_vs_cap'] = np.log(events['size_vs_cap'])
events.head()

,ACCESSION_NUMBER,side,dollar_value,date,log_dollar_value,role,ticker,is_10b5_1,P_t,P_t_plus_63,forward_ret_63,label_up,SP500_t,SP500_t_plus_63,forward_ret_SP500,label_up_market,market_cap,size_vs_cap,log_size_vs_cap
3,0000003570-25-000038,sell,5081343.54,2025-02-26,15.441086,OFFICER,LNG,0,217.482407,232.501556,0.069059,1,5956.060059,5921.540039,-0.005796,1,4.496016e+10,0.000113,-9.087956
4,0000003570-25-000040,sell,1975590.00,2025-02-27,14.496378,DIRECTOR,LNG,0,218.415924,230.700226,0.056243,1,5861.569824,5888.549805,0.004603,1,4.496016e+10,0.000044,-10.032665
5,0000004127-25-000028,buy,661300.00,2025-02-25,13.401964,OFFICER,SWKS,0,64.137993,69.934570,0.090377,1,5955.250000,5921.540039,-0.005661,1,9.374260e+09,0.000071,-9.559271
6,0000004977-25-000049,sell,2357419.89,2025-03-07,14.673079,OFFICER,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0,5.903470e+10,0.000040,-10.128313
7,0000004977-25-000051,sell,107630.00,2025-03-07,11.586464,DIRECTOR,AFL,1,106.673180,102.582268,-0.038350,0,5770.200195,5939.299805,0.029306,0,5.903470e+10,0.000002,-13.214937


In [49]:
events = events.drop(columns=['ACCESSION_NUMBER', 'date', 'dollar_value', 'ticker', 'P_t', 'P_t_plus_63', 'forward_ret_63', 'label_up', 'SP500_t', 'SP500_t_plus_63', 'forward_ret_SP500', 'market_cap', 'size_vs_cap'  ])

In [50]:
events.head(10)

,side,log_dollar_value,role,is_10b5_1,label_up_market,log_size_vs_cap
3,sell,15.441086,OFFICER,0,1,-9.087956
4,sell,14.496378,DIRECTOR,0,1,-10.032665
5,buy,13.401964,OFFICER,0,1,-9.559271
6,sell,14.673079,OFFICER,1,0,-10.128313
7,sell,11.586464,DIRECTOR,1,0,-13.214937
8,sell,15.554449,OFFICER,1,0,-9.246943
9,sell,13.269491,OFFICER,0,0,-10.009910
10,sell,15.226362,OFFICER,0,0,-8.053038
11,sell,13.603320,DIRECTOR,0,0,-9.676081
19,sell,11.336200,OFFICER,0,1,-11.597717


In [51]:
print(len(events))

5924


In [52]:
# Get current column order
cols = events.columns.tolist()

# Find indices of the columns to swap
idx_label = cols.index('label_up_market')
idx_log = cols.index('log_size_vs_cap')

# Swap their positions
cols[idx_label], cols[idx_log] = cols[idx_log], cols[idx_label]

# Reorder the DataFrame
events = events[cols]

# Verify the new order
print("New column order:")
print(events.columns.tolist())

New column order:
['side', 'log_dollar_value', 'role', 'is_10b5_1', 'log_size_vs_cap', 'label_up_market']


In [53]:
events.head(10)

,side,log_dollar_value,role,is_10b5_1,log_size_vs_cap,label_up_market
3,sell,15.441086,OFFICER,0,-9.087956,1
4,sell,14.496378,DIRECTOR,0,-10.032665,1
5,buy,13.401964,OFFICER,0,-9.559271,1
6,sell,14.673079,OFFICER,1,-10.128313,0
7,sell,11.586464,DIRECTOR,1,-13.214937,0
8,sell,15.554449,OFFICER,1,-9.246943,0
9,sell,13.269491,OFFICER,0,-10.009910,0
10,sell,15.226362,OFFICER,0,-8.053038,0
11,sell,13.603320,DIRECTOR,0,-9.676081,0
19,sell,11.336200,OFFICER,0,-11.597717,1


In [54]:
print(len(events))

5924


In [55]:
events.to_parquet("data/engineered/final_data_2025Q1.parquet") 